# 第5章 ベイズ推論プログラミング

## 5.1節 データ分布のベイズ推論

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp
#import jax.random as random

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 5.1 データ分布のベイズ推論
### 5.1.1 問題設定
```setosa```の```sepal_length```のデータ分布が正規分布に従うと仮定して、```sepal_length```の分布の形を調べる。

### 5.1.2 データ準備
```seaborn```の```iris```データセットを読み込む。\
```setosa```の萼片（がくへん）の長さ```sepal_length```を抽出する。\
```sepal_length```の分布に興味があるため目的変数の $y$ とする。

In [ ]:
# データセットを読み込む
df = sns.load_dataset("iris")

# setosa を抽出する
df_setosa = df.query('species == "setosa"')

# ヒストグラムを描画
bins = np.arange(4.0, 6.2, 0.2)
sns.histplot(data = df_setosa, x = "sepal_length", bins = bins, kde = True)
plt.xticks(bins);

In [ ]:
# NumPy変数の1次元配列に変換
Y = jnp.array(df_setosa['sepal_length'].values, dtype = float)

# 統計情報の確認
print(df_setosa['sepal_length'].describe())

# 値の確認
print(Y)

### 5.1.3 確率モデル定義

前章と同様に数式で整理してから確率モデルをプログラミングする。

1. ```setosa```の```sepal_length```のデータ分布が正規分布に従うと仮定する
    * $y_{i} \sim N(\mu, \sigma^2)$
1. 正規分布のパラメータに関する情報は無い。
    1. 正規分布の平均 $\mu$ が取りうる値はかなり広いものとする。
        * ヒストグラムより平均0, 標準偏差10の正規分布に従うと仮定する。
    1. 正規分布の標準偏差 $\sigma$ が取りうる値はかなり広いものとする。
        * 標準偏差10の半正規分布に従うと仮定する。

確率モデルは数式のまとめを終わりから実装していく。

In [ ]:
def model_normal(N, Y = None):
    '''
        5.1節のSetosaのがく片長さの確率分布モデル
    '''
    # 2.2. 正規分布の標準偏差 $\sigma$は標準偏差10の半正規分布に従うと仮定する。
    sigma = numpyro.sample("sigma", dist.HalfNormal(scale = 10))
    # 2.1. 正規分布の平均 $\mu$ はヒストグラムより平均0, 標準偏差10の正規分布に従うと仮定する。
    mu = numpyro.sample("mu", dist.Normal(loc = 0, scale = 10))
    # ベクトル化
    with numpyro.plate("N", N):
        # 1. $y_{i} \sim N(\mu, \sigma^2)$
        numpyro.sample("Y", dist.Normal(loc = mu, scale = sigma), obs = Y)

In [ ]:
model_args = {
    "N": len(Y),
    "Y": Y
}
try_render_model(model_normal, render_name = "Setosa's sepal length", **model_args)

### 5.1.4 サンプリング

In [ ]:
model_args = {
    "N": len(Y),
    "Y": Y
}
mcmc = run_mcmc(model_normal, num_chains = 4, num_warmup = 1000, num_samples = 1000, thinning = 1, seed = 42, **model_args)

### 5.1.5 結果分析

In [ ]:
az.plot_trace(mcmc, compact = False)
plt.tight_layout()

In [ ]:
ax = az.plot_posterior(mcmc)
plt.suptitle("Setosaのがく片長さの確率分布の形状")
plt.tight_layout()
plt.show()

In [ ]:
summary = az.summary(mcmc)
display(summary)

In [ ]:
print(f"mu={summary.loc["mu", "mean"]}, sigma={summary.loc["sigma", "mean"]}")

### 5.1.6 ヒストグラムと正規分布関数の重ね書き
ほぼ書籍のコードと同じ。

In [ ]:
def norm(x, mu, sigma):
    """
    正規分布のラインプロットの確率密度関数の値を計算する
    """
    y = (x-mu)/sigma
    a = np.exp(-(y**2)/2)
    b = np.sqrt(2*np.pi)*sigma
    return a/b

In [ ]:
x_min = Y.min()
x_max = Y.max()
x_list = np.arange(x_min, x_max, 0.01)
y_list = norm(x_list, summary.loc["mu", "mean"], summary.loc["sigma", "mean"])

In [ ]:
delta = 0.2
bins=np.arange(4.0, 6.0, delta)
fig, ax = plt.subplots()
sns.histplot(df_setosa, ax=ax, x='sepal_length',
    bins=bins, kde=True, stat='probability')
ax.get_lines()[0].set_label('KDE曲線')
ax.set_xticks(bins)
ax.plot(x_list, y_list*delta, c='r', label='ベイズ推論結果')
ax.set_title('ベイズ推論結果とKDE曲線の比較')
plt.legend();

### 5.1.7 少ないサンプル数でのベイズ推論
省略

### 